<h3>Парсинг объявлений о продаже автомобилей с Авито</h3>

In [1]:
# импорт библиотек
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 

import time
from random import randint

import pandas as pd

In [2]:
# начальная страница парсинга
URL = 'https://www.avito.ru/samarskaya_oblast/transport'

Напишем функцию для решения Captcha

In [3]:
def kill_captcha(driver):
    wait = WebDriverWait(driver, 5)
    try:
        # Проверка на наличие капчи
        captcha_element = wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//h2[@class='firewall-title']")), message="Капча обнаружена")

        if captcha_element:
            print("Капча обнаружена. Решите ее вручную.")
            input("Нажмите Enter, когда решите капчу...")

    except TimeoutException:
        pass

Напишем функцию получения данных с объявления 

In [4]:
def get_car_data(ad, mark):
    data = {}

    data['id'] = ad.get_attribute("id")
    data['mark'] = mark
    data['name'] = ad.find_element(
                        by=By.CSS_SELECTOR, value='h3[itemprop="name"]'
                    ).text

    data['price'] = ad.find_element(
                        by=By.CSS_SELECTOR, value='meta[itemprop="price"]'
                    ).get_attribute("content")

    data['params'] = ad.find_element(
                        by=By.CSS_SELECTOR, value='p[data-marker="item-specific-params"]'
                    ).text

    return  data

Напишем функцию парсинга для всех объявлений

In [5]:
def parse_cars(url):
    # Настраиваем параметры браузера
    options = webdriver.ChromeOptions()
    # Отключает функцию, указывающую сайтам, что браузер управляется автоматизацией
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)

    driver.get(url)

    kill_captcha(driver)
    # парсим популярные модели
    popular_models = driver.find_element(
        by=By.CSS_SELECTOR, value='div[data-marker="popular-rubricator/links"]'
    ).find_elements(
        by=By.CSS_SELECTOR, value='a[data-marker="popular-rubricator/link"]'
    )

    models = [{'name': x.text, 'link': x.get_attribute('href')} for x in popular_models]
    
    all_cars = []
    for model in models:
        # перейдем к объявлениям марки
        driver.get(model['link'])

        kill_captcha(driver)

        cars = []
        while True:
            # делаем небольшую задержку
            rand_sleep = randint(25, 49)
            time.sleep(rand_sleep / 10)

            ads = driver.find_elements(
                by=By.CSS_SELECTOR, value='div[data-marker="item"]'
            )

            for ad in ads:
                car_data = get_car_data(ad, model['name'])
                cars.append(car_data)
    
            try:
                # Ожидаем, пока кнопка станет видимой и доступной
                button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-marker="pagination-button/nextPage"]'))
                )

                if button.is_displayed() and button.is_enabled():
                    button.click()
                else:
                    break

            except Exception:
                break
                
        all_cars += cars
        pd.DataFrame(cars).to_csv(f"../data/brands/cars_data_{model['name']}.csv")
        
    driver.quit()
    return all_cars

*Авито* отображает максимум 100 страниц по 50 объявлений, поэтому мы проходимся по популярным брендам и парсим их по очереди, чтобы суммарно было больше объявлений.

In [7]:
# Парсинг
cars = parse_cars(URL)

In [8]:
# сохраняем данные
cars = pd.DataFrame(cars)

cars.to_csv('../data/cars_data.csv')

In [9]:
cars.sample(n=5)

,id,mark,name,price,params
4852,i3985280696,Geely,"Geely Emgrand 1.5 AT, 2024",2339990,"1.5 AT (122 л.с.), седан, передний, бензин"
6788,i4345701964,Hyundai,"Hyundai Solaris 1.4 MT, 2020, 96 984 км",1410000,"96 984 км, 1.4 MT (100 л.с.), седан, передний,..."
7690,i4259641901,Kia,"Kia Sportage 2.0 AT, 2024, 10 км",3900000,"10 км, 2 AT (150 л.с.), внедорожник, полный, б..."
6543,i4492345580,Hyundai,"Hyundai Creta 1.6 AT, 2018, 130 000 км",1750000,"130 000 км, 1.6 AT (123 л.с.), внедорожник, пе..."
6952,i3879412494,Hyundai,"Hyundai Solaris 1.4 MT, 2018, 148 140 км",1099990,"148 140 км, 1.4 MT (100 л.с.), седан, передний..."
